In [1]:
2+3

5

In [2]:
import h5py
import librosa
import numpy as np
import os
import pandas as pd
import random
import soundfile as sf
import sys
import time
import tqdm
import uuid



data_dir = '/beegfs/vl1019/BirdVox_datasets/'
predictions_dir = 'etc/BirdVox-full-night_baseline-predictions'

in_name = "BirdVox-full-night"
out_name = "BirdVox-70k"
in_sr = 24000.0
out_sr = 44100.0
in_dir = os.path.join(data_dir, in_name)
out_dir = os.path.join(data_dir, out_name)
os.makedirs(out_dir, exist_ok=True)
units = ["unit" + str(n).zfill(2) for n in [1, 2, 3, 5, 7, 10]]
hdf5_dir = os.path.join(out_dir, "hdf5_audio")
os.makedirs(hdf5_dir, exist_ok=True)

In [3]:
orig_sr = 24000
clip_length = int(0.500 * orig_sr) # a clip lasts 500 ms


for unit_str in units:
    
    print(unit_str)

    unit_path = os.path.join(predictions_dir,
        unit_str + ".npy")
    predictions = np.load(unit_path)

    samples = []
    annotation_name = "BirdVox-full-night_csv-annotations_" + unit_str + ".csv"
    annotations_dir = os.path.join(in_dir, "BirdVox-full-night_csv-annotations")
    annotation_path = os.path.join(annotations_dir, annotation_name)
    df = pd.read_csv(annotation_path, sep=',')
    recording_name = "BirdVox-full-night_flac-audio_" + unit_str + ".flac"
    recordings_dir = os.path.join(in_dir, "BirdVox-full-night_flac-audio")
    recording_path = os.path.join(recordings_dir, recording_name)
    full_night = sf.SoundFile(recording_path)
    n_positive_samples = 0
    n_negative_samples = 0
    samples = []


    # Load probabilities of the baseline prediction model.
    prediction_name = unit_str + ".npy"
    prediction_path = os.path.join(predictions_dir, prediction_name)
    prob_matrix = np.load(prediction_path)


    # Retrieve timestamps corresponding to decreasing confidences.
    prob_samples = (prob_matrix[:, 0] * orig_sr).astype('int')
    probs = prob_matrix[:, 1]
    sorting_indices = np.argsort(probs)[::-1]
    sorted_probs = probs[sorting_indices]
    sorted_prob_samples = prob_samples[sorting_indices]
    sorted_prob_samples = sorted_prob_samples


    # Initialize HDF5 container.
    file_name = "_".join(["BirdVox-70k", unit_str])
    file_path = os.path.join(hdf5_dir, file_name + ".hdf5")
    f = h5py.File(file_path, "w")
    waveform_group = f.create_group("waveforms")


    # Export every annotation either as positive (flight call).
    for index, row in tqdm.tqdm(df.iterrows()):
        # Compute center time of the annotation bounding box.
        mid_time = float(row["Time (s)"])
        sample = int(orig_sr * mid_time)
        samples.append(sample)
        sample_str = str(sample).zfill(9)

        # Compute center frequency of the annotation bounding box.
        mid_freq = float(row["Freq (Hz)"])
        freq_str = str(int(mid_freq)).zfill(5)

        # Define key.
        key = "_".join([unit_str, sample_str, freq_str, "1"])

        # Read.
        sample_start = sample - int(0.5 * clip_length)
        full_night.seek(sample_start)
        data = full_night.read(clip_length)
        waveform_group[key] = data

        n_positive_samples = n_positive_samples + 1


    # The exported false positives correspond to the timestamps with highest
    # confidences under the condition that they are 12000 samples (500 ms) apart
    # from all previously exported clips.
    prob_counter = 0
    false_positive_counter = 0
    n_false_positives = n_positive_samples
    while false_positive_counter < n_false_positives:
        prob_sample = sorted_prob_samples[prob_counter]
        dists = [np.abs(sample-prob_sample) for sample in samples]
        min_dist = np.min(dists)
        if min_dist > clip_length:
            # Append sample to growing list.
            sample_str = str(prob_sample).zfill(9)

            # Define key.
            # By convention, the frequency of a false positive example is 0 Hz.
            key = "_".join([unit_str, sample_str, "00000", "0"])

            # Read.
            sample_start = prob_sample - int(0.5 * clip_length)
            samples.append(prob_sample)
            full_night.seek(sample_start)
            data = full_night.read(clip_length)
            waveform_group[key] = data

            false_positive_counter = false_positive_counter + 1
        prob_counter = prob_counter + 1


    # Close file
    f.close()

unit01


2926it [00:02, 994.13it/s] 


unit02


4730it [00:07, 633.51it/s]


unit03


...

5264it [00:07, 694.76it/s] 


unit07


6530it [00:08, 800.10it/s]


unit10


6839it [00:09, 752.90it/s] 


In [15]:
os.system(''.join([
    'cp ',
    os.path.join(in_dir, 'BirdVox-full-night_gps-coordinates.csv'),
    ' ',
    os.path.join(out_dir, 'BirdVox-70k_gps-coordinates.csv'),
]))
os.system(''.join([
    'cp ',
    os.path.join(in_dir, 'BirdVox-full-night_utc-start-times.csv'),
    ' ',
    os.path.join(out_dir, 'BirdVox-70k_utc-start-times.csv'),
]))

0

In [13]:
''.join([
    'cp ',
    os.path.join(in_dir, 'BirdVox-full-night_gps-coordinates.csv'),
    os.path.join(out_dir, 'BirdVox-70k_gps-coordinates.csv'),
])

'cp /beegfs/vl1019/BirdVox_datasets/BirdVox-full-night/BirdVox-full-night_gps-coordinates.csv/beegfs/vl1019/BirdVox_datasets/BirdVox-70k/BirdVox-70k_gps-coordinates.csv'

In [5]:
from collections import Counter
f = h5py.File(file_path, "r")
print(Counter([x[-1] for x in list(f["waveforms"].keys())]))
f.close()

Counter({'0': 6839, '1': 6839})


In [31]:
n_positive_samples

5852

In [28]:
list(f["waveforms"].keys())

KeyError: "Unable to open object (Object 'waveforms' doesn't exist)"

In [23]:
list(f.keys())

[]

In [6]:
f.close()

In [29]:
list(f.keys())

[]

In [30]:
data

array([ -5.85937500e-03,   9.15527344e-05,  -6.50024414e-03, ...,
        -6.19506836e-03,   6.56127930e-03,   1.82495117e-02])

In [31]:
key

'unit01_942565200_04847_1'